**извлечение признаков из текста на естественном языке**

классификатор текстов

частотный анализ с хэшированием

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
2678,"Американские нейробиологи из Висконсинского университета в Мэдисоне, проанализировав, как функционирует мозг в процессе одновременного решения нескольких задач, сделали вывод, что в этом случае пр...",science
2735,"Пользователи сети пожаловались на рекламу в аудиозаписях во «ВКонтакте», которая теперь воспроизводится в конце композиций, загруженных в социальную сеть, и отмечается желтым маркером.\n\nПервыми ...",tech


##  HashingVectorizer

In [6]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    return ' '.join( [ t.strip() for t in tt if t ] )

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(n_features=16384, preprocessor=preprocessor, norm='l2')
vect.fit( data['text'])

HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=16384, ngram_range=(1, 1), non_negative=False,
         norm='l2', preprocessor=<function preprocessor at 0x7f8d8f40b1e0>,
         stop_words=None, strip_accents=None,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None)

## формируем датасеты

In [8]:
X = vect.transform(data['text'])
X.shape

(3196, 16384)

In [10]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [11]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [12]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [13]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 16384), (319,), (2877, 16384), (2877,))

## обучаем

In [14]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [15]:
o = clf.predict(X_train)

In [16]:
accuracy_score(y_train,o)

1.0

---

In [17]:
o = clf.predict(X_test)

In [18]:
accuracy_score(y_test,o)

0.7132429614181439

In [19]:
# print( classification_report(y_test,o) )

---

In [20]:
# from matplotlib import pyplot as plt
# import itertools

# classes = sorted(labels.keys())
# cm = confusion_matrix(y_test,o)
# tick_marks = np.arange(len(classes))

# plt.figure(figsize=(10,9))

# plt.xticks(tick_marks, classes, rotation=45)
# plt.yticks(tick_marks, classes)

# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], 'd'),
#              horizontalalignment="center",
#              color="white" if cm[i, j] > thresh else "black")

# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion matrix')
# plt.colorbar()
# plt.tight_layout()
# plt.ylabel('True label')
# plt.xlabel('Predicted label')

# plt.show()

---

In [21]:
o = clf.predict(X)

In [22]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [28]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: politics
predict: politics
- - - - - - - - - - - - - - - - - - 

Колинда Грабар-Китарович заявила, что на Балканах может начаться военный конфликт из-за России, которая активно влияет на события в регионе.

Глава МИД РФ ответил президенту Хорватии Колинде Грабар-Китарович на её слова о возможной российской угрозе на Балканах. По словам министра иностранных дел, её высказывания довольны странны.

— Вы знаете, я знаком с президентом Хорватии, у меня с ней, по-моему, даже нормальные отношения, с тех пор как она была ещё министром иностранных дел, и мне странно слышать такие слова из уст руководителя государства, с которым Россия хочет дружить, как и со всеми остальными государствами Балкан и на европейском континенте, — отметил Лавров в интервью Sputnik.

Министр также добавил, что если у Колинды Грабар-Китарович есть доказательства, то Москва готова обсудить с ней эти факты.

Напомним, хорватский лидер заявила, что на Балканах может начаться военный конфликт из-за России, которая ак